In [14]:
import itertools
import mdtraj as md
import mdtraj.testing



In [15]:
t = md.load('./data/pdbs/AF-C0H3V2-F1.pdb')
print(t)
hbonds = md.baker_hubbard(t)
for hbond in hbonds:
    print(hbond)

<mdtraj.Trajectory with 1 frames, 1105 atoms, 143 residues, without unitcells>


C:\Users\marik\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mdtraj\formats\pdb\pdbfile.py:206: UserWarning: Unlikely unit cell vectors detected in PDB file likely resulting from a dummy CRYST1 record. Discarding unit cell vectors.
  warnings.warn(


In [16]:
label = lambda hbond: f"{t.topology.atom(hbond[0])} -- {t.topology.atom(hbond[2])}"

for hbond in hbonds:
    print(label(hbond))

In [139]:
def pqr_parser(path, pdb_files=None):
    import numpy as np
    import os
    import scipy
    from scipy.spatial.distance import cdist
    
    if pdb_files is None:
        pdb_files = [f for f in os.listdir(path) if f.endswith('.pdb')]
    if isinstance(pdb_files, str):
        pdb_files = [pdb_files]
    for pdb_file in pdb_files:
    Donor_list = [('GLN', 'NE2', 'HE22'), ('GLN', 'NE2', 'HE23'), ('GLU', 'OE2', 'HE2'), ('ASP', 'OD2', 'HD2'), ('ASN', 'ND2', 'HD21'), ('ASN', 'ND2', 'HD22'),
                    ('HIS', 'NE2', 'HE2'), ('HIS', 'ND1', 'ND2'), ('LYS', 'NZ', 'HZ1'), ('LYS', 'NZ', 'HZ2'), ('LYS', 'NZ', 'HZ3'), ('ARG', 'NE', 'HE'), 
                    ('ARG', 'NH1', 'HH11'), ('ARG', 'NH1', 'HH12'), ('ARG', 'NH2', 'HH21'), ('ARG', 'NH2', 'HH22'), ('SER', 'OG', 'HG'), ('THR', 'OG1', 'HG1'), 
                    ('TRP', 'NE1', 'HE1'), ('TYR', 'OH', 'HH')]
    Acceptor_list = [('GLN', 'OE1'), ('ASP', 'OD1'), ('ASP', 'OD2'), ('ASN', 'OD1')]
    with open(os.path.join(path, 'C0H3Z2.pqr')) as f:
        Donor_array = np.empty((0, 4))
        H_array = np.empty((0, 4))
        Acceptor_array = np.empty((0, 4))
        aa_cache = []
        atom_cache = []
        for line in f:
            if line.startswith('ATOM'):
                if not aa_cache:
                    aa_cache.append(line.split()[3])
                    atom_cache.append(line)
                elif aa_cache[0] == line.split()[3]:
                    atom_cache.append(line)
                else:
                    if aa_cache[0] != line.split()[3]:                
                        for n in range(len(Donor_list)):
                            if aa_cache[0] == Donor_list[n][0]:
                                for i in range(len(atom_cache)):
                                    if Donor_list[n][1] == atom_cache[i].split()[2]:
                                        if ('GLU' == aa_cache[0]) and 'OE2' in atom_cache[i].split()[2]:
                                            if any('HE2' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0) 
                                        elif ('ASP' == aa_cache[0]) and 'OD2' in atom_cache[i].split()[2]:
                                            if any('HD2' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0)                                             
                                        elif('HIS' == aa_cache[0]) and 'ND1' in atom_cache[i].split()[2]:
                                            if any ('HD1' in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Donor_array = np.append(Donor_array, line_array, axis=0)
                                        else: 
                                            line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                            line_array = line_array.astype('float64')
                                            Donor_array = np.append(Donor_array, line_array, axis=0)         
                                    elif Donor_list[n][2] == atom_cache[i].split()[2]:
                                        line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                        line_array = line_array.astype('float64')
                                        H_array = np.append(H_array, line_array, axis=0)
                        for n in range(len(Acceptor_list)):
                            if aa_cache[0] == Acceptor_list[n][0]:
                                for i in range(len(atom_cache)):
                                    if Acceptor_list[n][1] == atom_cache[i].split()[2]:
                                        if ('GLU' == aa_cache[0]) and 'OE2' in atom_cache[i].split()[2]:
                                            if any('HE2' not in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Acceptor_array = np.append(Acceptor_array, line_array, axis=0)
                                        elif ('ASP' == aa_cache[0]) and 'OD' in atom_cache[i].split()[2]:
                                            if any('HD' not in string for string in atom_cache):
                                                line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                                line_array = line_array.astype('float64')
                                                Acceptor_array = np.append(Acceptor_array, line_array, axis=0)
                                        else: 
                                            line_array = np.array([[atom_cache[i].split()[1], atom_cache[i].split()[5], atom_cache[i].split()[6], atom_cache[i].split()[7]]])
                                            line_array = line_array.astype('float64')
                                            Donor_array = np.append(Donor_array, line_array, axis=0)
    return Donor_array, H_array, Acceptor_array



[[ 3.9000e+01 -1.4120e+01 -6.0410e+00  8.5210e+00]
 [ 4.0000e+01 -1.3733e+01 -4.4660e+00  8.3300e+00]
 [ 4.1000e+01 -1.5167e+01 -4.8790e+00  8.9930e+00]
 [ 7.7000e+01 -1.3169e+01 -4.5550e+00  2.2840e+00]
 [ 1.4800e+02  5.9300e-01  8.5840e+00 -7.9420e+00]
 [ 1.4900e+02 -2.6470e+00  8.5840e+00 -9.3750e+00]
 [ 1.5000e+02 -2.0450e+00  9.0810e+00 -1.0921e+01]
 [ 1.5100e+02  1.2530e+00  9.3270e+00 -9.9250e+00]
 [ 1.5200e+02  1.2200e-01  9.4950e+00 -1.1225e+01]
 [ 2.1300e+02  1.1978e+01  5.4650e+00 -7.0490e+00]
 [ 2.2700e+02  1.3337e+01  9.5800e+00 -1.2762e+01]
 [ 2.6400e+02  1.9865e+01  5.4070e+00 -8.4590e+00]
 [ 2.8800e+02  1.1538e+01  3.6110e+00 -1.4134e+01]
 [ 3.1200e+02  1.6392e+01  1.1210e+00 -5.2740e+00]
 [ 2.6500e+02  1.9435e+01  4.5550e+00 -5.0460e+00]
 [ 2.8900e+02  8.1220e+00  3.3140e+00 -1.3284e+01]
 [ 3.1300e+02  1.6922e+01  2.9940e+00 -2.3210e+00]
 [ 2.6600e+02  2.1016e+01  5.1200e+00 -4.6220e+00]
 [ 2.9000e+02  7.6010e+00  3.7950e+00 -1.4865e+01]
 [ 3.1400e+02  1.8404e+01  2.17

In [92]:
Donor_list = [('GLN', 'NE2', 'HE22', 'HE23'), ('GLU', 'OE2', 'HE2'), ('ASP', 'OD2', 'HD2'), ('ASN', 'ND2', 'HD21', 'HD22'), 
                  ('HIS', 'NE2', 'HE2'), ('HIS', 'ND1', 'ND2'), ('LYS', 'NZ', 'HZ1', 'HZ2', 'HZ3'), ('ARG', 'NE', 'HE'), 
                  ('ARG', 'NH1', 'HH11', 'HH12'), ('ARG', 'NH2', 'HH21', 'HH22'), ('SER', 'OG', 'HG'), ('THR', 'OG1', 'HG1'), 
                  ('TRP', 'NE1', 'HE1'), ('TYR', 'OH', 'HH')]
for i in range(len(Donor_list[3])-2):
    print(i)


0
1


In [134]:
string = 'ATOM     12  HB3 MET     1     -20.987   0.463   6.535  0.0000 0.0000'
string2 ='ATOM      5  O   MET     1     -18.415  -1.135   7.336 -0.5500 1.4000'
list2 = [string, string2]
print(list2)

if 'MET' in list2:
    print('yes')
else:
    print('no')


['ATOM     12  HB3 MET     1     -20.987   0.463   6.535  0.0000 0.0000', 'ATOM      5  O   MET     1     -18.415  -1.135   7.336 -0.5500 1.4000']
no
